In [1]:
import requests,time,datetime
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu
import utils.StockSpider as ss

In [2]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [3]:
df = df_hsA

In [4]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# ss.getAnnualReports(dir_path,df,'2020')

# ss.getStockDataByType('E:/wangtao/PythonWorkSpace/DataAnalysis/StockAnalyze/data/')

# ss.getStockDataByType('E:/PythonWorkSpace/DataAnalysis/Stock_Companies/data/')


In [5]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [6]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})
df_2019 = pd.read_excel("data/2019年报.xls",converters= {u'stock_code':str})
df_2020 = pd.read_excel("data/2020年报.xls",converters= {u'stock_code':str})

In [7]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于?%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.15) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0) & \
            (row['operate_cash_flow_ps'] > 0):
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [8]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [9]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [10]:
df_target.shape

(10, 19)

In [11]:
df_target = df_target[df_target['PE_D'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
df_target = df_target[df_target['PE_S'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:ss.getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_S']) / x['PE_M']),axis=1)
# df_target = df_target[df_target['pe_rate'] < 0.6]

In [12]:
df_target.shape

(10, 21)

In [15]:
df_target.sort_values(by=['pe_rate'],ascending=True)

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare,PE_M,pe_rate
1152,002841,视源股份,80.62,2.90%,1.15,0.39%,电子 — 光学光电子,7.72,28.26,62.485,0.66元,4.30亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降20.51%\t...,5.79%,24.46%,10.44元,6.67亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,-0.15元,42.750,0.661053
1495,603658,安图生物,54.57,2.38%,1.19,1.14%,医药生物 — 医疗器械,4.77,42.78,38.763,0.70元,4.13亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长50.63%\t...,6.24%,59.89%,11.45元,5.86亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.89元,54.370,0.786831
117,600276,恒瑞医药,54.19,7.88%,1.73,1.76%,医药生物 — 化学制药,10.58,54.78,64.980,0.42元,26.68亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长0.21%\t...,8.34%,86.81%,5.12元,63.97亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,0.23元,68.225,0.802931
369,603180,金牌厨柜,31.83,5.12%,1.40,1.05%,轻工制造 — 家居用品,2.26,16.79,28.317,0.56元,0.87亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长41.15%\t...,4.62%,27.73%,14.07元,1.54亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,NaN,-0.12元,20.355,0.824859
951,002508,老板电器,34.90,3.25%,1.21,1.14%,家用电器 — 厨卫电器,4.07,19.94,20.952,0.83元,7.90亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长29.08%\t...,9.49%,56.38%,8.58元,9.49亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.56元,21.680,0.919742
783,603899,晨光文具,70.45,3.63%,0.98,0.44%,轻工制造 — 文娱用品,12.54,52.06,49.053,0.72元,6.66亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长43.43%\t...,12.19%,24.01%,5.62元,9.28亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.39元,42.080,1.237167
377,600887,伊利股份,39.62,5.09%,1.57,2.28%,食品饮料 — 饮料乳品,7.48,34.05,22.640,0.88元,53.22亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长42.48%\...,15.90%,37.40%,5.30元,60.82亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,1.04元,26.330,1.293202
1279,002415,海康威视,56.48,2.69%,1.38,0.47%,计算机 — 计算机设备,9.96,39.39,40.677,0.69元,64.81亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长40.17%\...,11.62%,46.30%,5.67元,93.36亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,0.21元,28.790,1.368183
2007,603288,海天味业,112.00,1.82%,0.68,0.22%,食品饮料 — 调味发酵品Ⅱ,23.48,73.69,70.364,0.80元,33.53亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长3.07%\t...,16.37%,39.31%,4.77元,42.13亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,0.06元,52.310,1.408717
3092,600519,贵州茅台,1839.60,0.52%,0.72,0.38%,食品饮料 — 白酒Ⅱ,14.29,49.49,46.867,19.63元,246.54亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长9.08%\...,14.20%,91.38%,128.75元,12.56亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,17.29元,33.225,1.489541


,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare
2822,000157,中联重科,8.33,0.97%,1.09,1.19%,机械设备 — 工程机械,1.31,9.93,7.452,0.59元,48.50亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长20.7%\t...,8.93%,25.01%,6.38元,86.78亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,NaN,0.48元


In [16]:
# df.sort_values(by=['Price'],ascending=False)

In [32]:
# df_target
n = '三一重工'

In [33]:
df[df['StockName'] == n ]

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare
1197,600031,三一重工,26.16,2.83%,1.08,1.42%,机械设备 — 工程机械,3.59,14.40,11.027,1.20元,100.74亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长17.16%...,16.34%,28.14%,7.29元,84.92亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t超大盘股\t\t\t\t\t\t\t...,NaN,1.15元
